In [181]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters, haversine
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name
from agile.utils.dataframes import modify_and_sort_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

In [169]:
with open('saved_data/saved_df.pkl', 'rb') as file:
    df = pickle.load(file)

In [174]:
adid = '532a2b51-0444-3e09-648c-b54d88d7d282'
num_clusters = 4

In [176]:
cluster_data = double_cluster(adid, df)
loi_data = get_top_N_clusters(cluster_data, num_clusters)
loi_data_i = pd.DataFrame()
loi_data

,latitude,longitude,datetime,geohash,advertiser_id,label
label,,,,,,
16,46.260270,-119.249642,2018-04-08 00:04:45.000,c25pnmmj9g,532a2b51-0444-3e09-648c-b54d88d7d282,29
18,46.258200,-119.255310,2018-04-08 00:03:50.000,c25pnm0ctc,532a2b51-0444-3e09-648c-b54d88d7d282,18
28,46.262476,-119.245646,2018-04-08 00:05:26.500,c25pnmzfrn,532a2b51-0444-3e09-648c-b54d88d7d282,16
29,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,28


loi_data['iter'] = [0] * len(loi_data)

In [199]:
cluster_data = pd.DataFrame()
loi_data = pd.DataFrame()
loi_data_i = pd.DataFrame()

for i in range(1,10):
    print(i)
    cluster_data = double_cluster(adid, df)
    loi_data_i = get_top_N_clusters(cluster_data, i)
    loi_data_i['i'] = [i] * len(loi_data_i)
    loi_data = pd.concat([loi_data, loi_data_i])
    for j in range(1,13):
        print(j)
        
        #colocation = pd.concat([colocation, colocation(df, loi_data, duration=j)])
    
loi_data

1
1
2
3
4
5
6
7
8
9
10
11
12
2
1
2
3
4
5
6
7
8
9
10
11
12
3
1
2
3
4
5
6
7
8
9
10
11
12
4
1
2
3
4
5
6
7
8
9
10
11
12
5
1
2
3
4
5
6
7
8
9
10
11
12
6
1
2
3
4
5
6
7
8
9
10
11
12
7
1
2
3
4
5
6
7
8
9
10
11
12
8
1
2
3
4
5
6
7
8
9
10
11
12
9
1
2
3
4
5
6
7
8
9
10
11
12


,latitude,longitude,datetime,geohash,advertiser_id,label,i
label,,,,,,,
29,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,29,1
21,46.250790,-119.254857,2018-04-08 00:02:04.000,c25pn79q2k,532a2b51-0444-3e09-648c-b54d88d7d282,29,2
29,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,21,2
12,46.250941,-119.254953,2018-04-08 00:02:07.500,c25pn79pq0,532a2b51-0444-3e09-648c-b54d88d7d282,26,3
13,46.250183,-119.254472,2018-04-08 00:02:01.000,c25pn79dc6,532a2b51-0444-3e09-648c-b54d88d7d282,13,3
26,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,12,3
0,46.253285,-119.257001,2018-04-08 00:02:36.000,c25pnhpsym,532a2b51-0444-3e09-648c-b54d88d7d282,24,4
9,46.250790,-119.254857,2018-04-08 00:02:04.000,c25pn79q2k,532a2b51-0444-3e09-648c-b54d88d7d282,10,4
10,46.248824,-119.253226,2018-04-08 00:01:44.000,c25pn766ym,532a2b51-0444-3e09-648c-b54d88d7d282,9,4


In [200]:
def filter_close_coordinates(df, threshold_distance=0.1):
    new_df = pd.DataFrame(columns=df.columns)

    for i, row1 in df.iterrows():
        keep_row = True
        for _, row2 in new_df.iterrows():
            distance = haversine(row1['latitude'], row1['longitude'], row2['latitude'], row2['longitude'])
            if distance < threshold_distance:
                keep_row = False
                break
        if keep_row:
            new_df = new_df.append(row1, ignore_index=True)

    return new_df

In [204]:
loi_data = filter_close_coordinates(loi_data, threshold_distance=.5)
loi_data

/tmp/ipykernel_65218/4210164782.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row1, ignore_index=True)


,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,29,1
1,46.250790,-119.254857,2018-04-08 00:02:04.000,c25pn79q2k,532a2b51-0444-3e09-648c-b54d88d7d282,29,2
2,46.261612,-119.247414,2018-04-08 00:05:09.500,c25pnmwtmv,532a2b51-0444-3e09-648c-b54d88d7d282,15,5
3,46.244381,-119.248898,2018-04-08 00:00:46.500,c25pn6t953,532a2b51-0444-3e09-648c-b54d88d7d282,6,5
4,46.255921,-119.258800,2018-04-08 00:03:07.500,c25pnhwkk8,532a2b51-0444-3e09-648c-b54d88d7d282,8,9


In [205]:
loi_data.drop_duplicates(subset=['latitude','longitude'])

,latitude,longitude,datetime,geohash,advertiser_id,label,i
0,46.275998,-119.194395,2018-04-08 15:09:54.000,c25pr8m31r,532a2b51-0444-3e09-648c-b54d88d7d282,29,1
1,46.250790,-119.254857,2018-04-08 00:02:04.000,c25pn79q2k,532a2b51-0444-3e09-648c-b54d88d7d282,29,2
2,46.261612,-119.247414,2018-04-08 00:05:09.500,c25pnmwtmv,532a2b51-0444-3e09-648c-b54d88d7d282,15,5
3,46.244381,-119.248898,2018-04-08 00:00:46.500,c25pn6t953,532a2b51-0444-3e09-648c-b54d88d7d282,6,5
4,46.255921,-119.258800,2018-04-08 00:03:07.500,c25pnhwkk8,532a2b51-0444-3e09-648c-b54d88d7d282,8,9


In [206]:
# Create a Folium map centered around the mean coordinates
map_center = [loi_data['latitude'].mean(), loi_data['longitude'].mean()]
folium_map = folium.Map(location=map_center, zoom_start=5)

# Add markers for each row in the DataFrame
for index, row in loi_data.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']]).add_to(folium_map)
    
folium_map

In [ ]:
colocation = colocation(df, loi_data, duration=12)

Empty DataFrame
Columns: [geohash, datetime, latitude, longitude, advertiser_id]
Index: []
